# crewai-poemflow
By [Zia Khan](https://www.linkedin.com/in/ziaukhan/)

Github repo: https://github.com/panaversity/learn-agentic-ai/tree/main/05_crew_ai

Docs: https://docs.crewai.com/concepts/flows

Simplified version of a **flows** CrewAI which is giving an error when I run it.

In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain-google-genai langchain_community --quiet
print("---")
%pip show crewAI crewai_tools langchain-google-genai langchain_community

In [ ]:
# @title 🔑 Import libs and get API Key by running this cell

import os
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from random import randint
from pydantic import BaseModel
from crewai.flow.flow import Flow, listen, start
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Check if the 'output-files' directory exists, and create it if it doesn't
if not os.path.exists('output-files'):
    os.makedirs('output-files')

In [ ]:
# @title 🕵🏻 Define your crews, agents and tasks


def createPoemCrew()->Crew:

    # Initialize the Gemini LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        verbose=True,
        temperature=0.5,
        google_api_key=GEMINI_API_KEY
    )


    #https://docs.crewai.com/concepts/agents#creating-an-agent
    poem_writer: Agent = Agent(
    role='CrewAI Poem Writer',
    goal="""Generate a funny, light heartedpoem about how CrewAI is
            awesome with a sentence count of {sentence_count}""",
    backstory="""You're a creative poet with a talent for capturing the essence of any topic
        in a beautiful and engaging way. Known for your ability to craft poems that
        resonate with readers, you bring a unique perspective and artistic flair to
        every piece you write.""",
    llm=llm,
    allow_delegation=True,
    verbose=True,
    memory=True
    )

    #https://docs.crewai.com/concepts/tasks#creating-a-task
    write_poem: Task = Task(
        description="""Write a poem about how CrewAI is awesome.
        Ensure the poem is engaging and adheres to the specified
        sentence count of {sentence_count}.""",
        agent=poem_writer,
        expected_output='A beautifully crafted poem about CrewAI, with exactly {sentence_count} sentences.',
    )

    #https://docs.crewai.com/concepts/crews
    return Crew(
                agents=[poem_writer], # Automatically created by the @agent decorator
                tasks=[write_poem], # Automatically created by the @task decorator
                process=Process.sequential,
                verbose=True,
            )


In [ ]:
# @title 📝 Define your Flows and State
# Task Definitions

class PoemState(BaseModel):
    sentence_count: int = 1
    poem: str = ""


class PoemFlow(Flow[PoemState]):

    @start()
    def generate_sentence_count(self):
        print("Generating sentence count")
        self.state.sentence_count = randint(1, 5)

    @listen(generate_sentence_count)
    def generate_poem(self):
        print("Generating poem")
        result = (
            createPoemCrew()
            .kickoff(inputs={"sentence_count": self.state.sentence_count})
        )

        print("Poem generated", result.raw)
        self.state.poem = result.raw

    @listen(generate_poem)
    def save_poem(self):
        print("Saving poem")
        with open("poem.txt", "w") as f:
            f.write(self.state.poem)



In [ ]:
# @title ⌨️ Define your kickoff and plot functions

def kickoff():
    poem_flow = PoemFlow()
    return poem_flow.kickoff()


def plot():
    poem_flow = PoemFlow()
    poem_flow.plot()



In [ ]:
# @title 🚀 Get your flow to work and print final output

final_output = kickoff()

print("---- Final Output ----")
print(final_output)

RuntimeError: asyncio.run() cannot be called from a running event loop

The error message RuntimeError: asyncio.run() cannot be called from a running event loop is occurring because you are trying to use asyncio.run() within a Jupyter Notebook environment, which already has a running event loop. asyncio.run() is designed to be the entry point for asynchronous programs and is not intended to be called from within an existing event loop.

In the code, within the crewai.flow.flow.py module's kickoff function, asyncio.run() is used to execute the kickoff_async function. Since Jupyter notebooks already run an asyncio event loop in the background, this creates a conflict resulting in the observed error.

In [ ]:
!crewai flow kickoff

In [ ]:
!uv run kickoff

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm